In [1]:
#hide
#skip
%config Completer.use_jedi = False
%config IPCompleter.greedy=True
# upgrade fastrl on colab
! [ -e /content ] && pip install -Uqq fastrl['dev'] pyvirtualdisplay && \
                     apt-get install -y xvfb python-opengl > /dev/null 2>&1 
# NOTE: IF YOU SEE VERSION ERRORS, IT IS SAFE TO IGNORE THEM. COLAB IS BEHIND IN SOME OF THE PACKAGE VERSIONS

In [2]:
# hide
from fastcore.imports import in_colab
# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbverbose.showdoc import *
    from nbdev.imports import *
    if not os.environ.get("IN_TEST", None):
        assert IN_NOTEBOOK
        assert not IN_COLAB
        assert IN_IPYTHON
else:
    # Virutual display is needed for colab
    from pyvirtualdisplay import Display
    display = Display(visible=0, size=(400, 300))
    display.start()

In [3]:
# default_exp fastai.data.load.alt

In [4]:
# export
# Python native modules
import os
# Third party libs
from fastcore.all import *
from fastai.torch_basics import *
# from torch.utils.data.dataloader import DataLoader as OrgDataLoader
from torch.utils.data import dataloader
# Local modules
from fastrl.fastai.loop import *

from fastai.data.load import DataLoader as FastaiDataLoader

from torchdata.datapipes.iter import HttpReader

# DataLoader helpers
> An implimentation of the torch DataLoader that instead uses Loop as a base. It
also uses experimental API from torch.data

In [5]:
bs = 4
letters = list(string.ascii_lowercase)

In [6]:
# export
@events
class DataLoader(dataloader.DataLoader,Loop):
    def __init__(self,datapipe,*args,bs=1,n_workers=0,**kwargs):        
        Loop.__init__(self)
        
        # connect_loops2loop((BatchLoop()),self)
        
        self.it=None
        self.current_batch=None
        self.iter_section=self.sections['iter']
        self.run(['create'])

        dataloader.DataLoader.__init__(self,datapipe,*args,batch_size=bs,num_workers=n_workers,**kwargs)
        
    def on_create(self):pass
        
    def on_iter(self):
        self.it=dataloader.DataLoader.__iter__(self)
        
    def __iter__(self):
        try:
            self.iter_section['before_']()
            self.iter_section['before_inner'].run()                             # fastrl.skip_traceback
            self.iter_section['on_']()
            yield from self.it
            self.iter_section['after_']()
            self.iter_section['after_inner'].run()                              # fastrl.skip_traceback
        except Exception as ex:
            try:
                self.iter_section['failed_']()                                  # fastrl.skip_traceback
                raise
            finally:
                self.iter_section['failed_inner'].run()                         # fastrl.skip_traceback
        finally:
            self.iter_section['finally_']()
            self.iter_section['finally_inner'].run()                            # fastrl.skip_traceback

# class LoopyIter():
    
    
            
@events
class BatchLoop(Loop):
    call_on=L(DataLoader.on_iter)
    
    def __init__(self):
        
        super().__init__(self)
    
    @property
    def dl(self): return self.root_loop

    def on_create_batches(self):pass
        

In [8]:
for o in DataLoader(letters,bs=2): print(o)

['a', 'b']
['c', 'd']
['e', 'f']
['g', 'h']
['i', 'j']
['k', 'l']
['m', 'n']
['o', 'p']
['q', 'r']
['s', 't']
['u', 'v']
['w', 'x']
['y', 'z']


In [ ]:
# hide
from fastcore.imports import in_colab

# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev.export import *
    from nbdev.export2html import *
    from nbverbose.cli import *
    make_readme()
    notebook2script()
    notebook2html()